In [1]:
#https://www.tensorflow.org/get_started/mnist/beginners

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
# TensorFlow Graph visualizer code
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))



In [ ]:
plt.imshow(np.reshape(mnist.train.images[0],(28,28)))
plt.show()

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])

In [ ]:
with tf.device('/gpu:0'):
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    #W = tf.Variable(tf.random_normal([784, 10]))
    #b = tf.Variable(tf.random_normal([10]))

In [ ]:
#y = tf.nn.softmax(tf.matmul(x, W) + b)
with tf.device('/gpu:0'):
    y = tf.nn.softmax(tf.tanh(tf.matmul(x, W) + b)) #tf.tanh

In [ ]:
#True classes
with tf.device('/gpu:0'):
    y_ = tf.placeholder(tf.float32, [None, 10])

In [ ]:
#cross_entropy
with tf.device('/gpu:0'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [ ]:
with tf.device('/gpu:0'):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [ ]:
#Session executing model
#workarounds for GPU memory in jupyter notebook
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
config = tf.ConfigProto(log_device_placement=True, gpu_options=gpu_options)
config.gpu_options.allow_growth=True
sess = tf.InteractiveSession(config=config)
tf.global_variables_initializer().run()

In [ ]:
accs = []
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(10000)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    #cp = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    #acc = tf.reduce_mean(tf.cast(cp, tf.float32))
    #accs.append(acc)

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

In [ ]:
print("W=")
print(sess.run(W))
print("b=")
print(sess.run(b))

In [ ]:
#plot(accs)

In [4]:
#Session executing model
#workarounds for GPU memory in jupyter notebook
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
config = tf.ConfigProto(log_device_placement=True, gpu_options=gpu_options)
config.gpu_options.allow_growth=True
sess = tf.InteractiveSession(config=config)
tf.global_variables_initializer().run()

In [15]:
hidden_size = 784 #* 5
g = tf.Graph()
#with g.as_default():
with tf.device('/gpu:0'):
    x = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.float32, [None, 10])

    #W1 = tf.Variable(tf.ones([784, hidden_size]))
    W1 = tf.Variable(tf.eye(784))
    #W1 = tf.Variable(tf.zeros([784, hidden_size], trainable=False))
    W2 = tf.Variable(tf.zeros([hidden_size, 10]))
    b1 = tf.Variable(tf.zeros([hidden_size]))
    b2 = tf.Variable(tf.zeros([10]))

    hid = (tf.sigmoid(tf.matmul(x, W1) + b1)) #tf.tanh tf.sigmoid
    y = tf.nn.softmax(tf.sigmoid(tf.matmul(hid, W2) + b2)) #tf.tanh tf.sigmoid

    #W1 = tf.Variable(tf.zeros([784, 10]))
    #y = tf.nn.softmax(tf.tanh(tf.matmul(x, W1) + b2))
    
    #y = tf.nn.softmax(tf.tanh(tf.matmul(x, W2) + b2))

    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
    #err = (y_ - y)**2
    #train_step = tf.train.GradientDescentOptimizer(0.1).minimize(err)
tf.global_variables_initializer().run()

In [31]:
accs = []
for _ in range(10000): #1000
    batch_xs, batch_ys = mnist.train.next_batch(1000) #30000
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [32]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9042


In [33]:
#print(tf.trainable_variables())
print("W1=")
print(sess.run(W1))
print("W2=")
print(sess.run(W2))
print("b1=")
print(sess.run(b1))
print("b2=")
print(sess.run(b2))

W1=
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
W2=
[[-0.04534516  0.06636716 -0.03111922 ...,  0.02294132 -0.06148178
   0.00117276]
 [-0.04534516  0.06636716 -0.03111922 ...,  0.02294132 -0.06148178
   0.00117276]
 [-0.04534516  0.06636716 -0.03111922 ...,  0.02294132 -0.06148178
   0.00117276]
 ..., 
 [-0.04534516  0.06636716 -0.03111922 ...,  0.02294132 -0.06148178
   0.00117276]
 [-0.04534516  0.06636716 -0.03111922 ...,  0.02294132 -0.06148178
   0.00117276]
 [-0.04534516  0.06636716 -0.03111922 ...,  0.02294132 -0.06148178
   0.00117276]]
b1=
[  1.11099426e-03   1.11099426e-03   1.11099426e-03   1.11099426e-03
   1.11099426e-03   1.11099426e-03   1.11099426e-03   1.11099426e-03
   1.11099426e-03   1.11099426e-03   1.11099426e-03   1.11099426e-03
   1.11101824e-03   1.11091300e-03   1.11041719e-03   1.11094466e-03
   1.1109942

In [ ]:
show_graph(g)